In [2]:
import numpy as np
import pandas as pd
import astropy.units as u
from astropy.coordinates import SkyCoord
from astroquery.gaia import Gaia
from astroquery.simbad import Simbad
from astroquery.vizier import Vizier
from typing import Callable, Dict
import os
import glob

from auxiliary.filter_functions import filter_gaia, filter_archival, filter_chandra, filter_erosita, filter_ned, filter_simbad, filter_vizier, filter_Xray_binaries


In [ ]:

def safe_read_filtered(filename: str, catalogs: Dict[str, Callable], detections: pd.DataFrame = None) -> pd.DataFrame:
    if os.path.exists(filename):
        return pd.read_csv(filename, sep=',', header=0, dtype=str)
    else:
        if detections is not None:
            # use detections columns as a template
            df = pd.DataFrame(columns=detections.columns)
        else:
            # fallback: minimal structure
            df = pd.DataFrame(columns=['Number','ObsId','RA','DEC','THETA','POS_ERR','SIGNIFICANCE'])
        return update_catalogs(df, catalogs)


def update_catalogs(dataframe: pd.DataFrame, catalogs: Dict[str, Callable], verbose: int = 0) -> pd.DataFrame:
    dataframe = dataframe.copy()

    for catalog, _ in catalogs.items():
        if f'{catalog}_match' not in dataframe.columns:
            dataframe.insert(len(dataframe.columns),
                             f'{catalog}_match', 'unknown')

            if verbose > 2:
                print(f'Added {catalog}_match column.')

    return dataframe


def update_detections(detections: pd.DataFrame, filtered: pd.DataFrame, catalogs: Dict[str, Callable], verbose: int = 0) -> pd.DataFrame:
    detections = detections.copy()
    filtered = filtered.copy()

    detections = update_catalogs(detections, catalogs)
    filtered = update_catalogs(filtered, catalogs)

    # add new detections to filtered dataframe
    for i, detection in detections.iterrows():
        if (
            detection.at['ObsId'] in filtered['ObsId'].values and
            detection.at['RA'] in filtered['RA'].values and
            detection.at['DEC'] in filtered['DEC'].values and
            detection.at['THETA'] in filtered['THETA'].values and
            detection.at['POS_ERR'] in filtered['POS_ERR'].values and
            detection.at['SIGNIFICANCE'] in filtered['SIGNIFICANCE'].values
        ):
            if verbose > 2:
                print(f'{i}: {detection.at["ObsId"]} - Detection already in filtered.')
            continue

        filtered.loc[len(filtered)] = detection

        if verbose > 2:
            print(f'{i}: {detection.at["ObsId"]} - Added detection to filtered.')

    return filtered


def load_all_filtered_refs(catalogs: Dict[str, Callable], detections: pd.DataFrame) -> pd.DataFrame:
    """
    ## Combine all existing filtered_w*_forward.csv files into one dataframe.
    This lets the script check detections across all time windows.
    """
    files = glob.glob("output/filtered_w*_forward.csv")
    all_refs = []

    for file in files:
        try:
            df = safe_read_filtered(file, catalogs, detections)
            df["source_file"] = file  # Track which file it came from
            all_refs.append(df)
        except Exception as e:
            print(f"Skipping {file} ({e})")

    if not all_refs:
        return pd.DataFrame(columns=detections.columns)

    combined = pd.concat(all_refs, ignore_index=True).drop_duplicates(subset=["RA", "DEC"])
    print(f"Loaded {len(combined)} total reference detections from {len(all_refs)} files.")
    return combined


def filter_detections(detections: pd.DataFrame, filtered: pd.DataFrame, catalogs: Dict[str, Callable], verbose: int = 0) -> pd.DataFrame:
    # add new detections to filtered dataframe
    filtered = update_detections(detections, filtered, catalogs, verbose)

    # combine all filtered reference files (across all window sizes)
    filtered_ref = load_all_filtered_refs(catalogs, detections)

    for i, detection in filtered.iterrows():
        # check if detection is in the combined reference dataframe
        if detection['RA'] in filtered_ref['RA'].values and detection['DEC'] in filtered_ref['DEC'].values:
            filtered_detection = filtered_ref[(filtered_ref['RA'] == detection['RA']) &
                                              (filtered_ref['DEC'] == detection['DEC'])].iloc[0]
            for catalog in catalogs.keys():
                if filtered_detection[f'{catalog}_match'] != 'unknown':
                    filtered.at[i, f'{catalog}_match'] = filtered_detection[f'{catalog}_match']
            if verbose > 1:
                print(f'{i}: {detection.at["ObsId"]} - populated from previous filtered files.')
            continue

        for catalog, filter_func in catalogs.items():
            if detection.at[f'{catalog}_match'] == 'unknown':
                try:
                    result = 'yes' if filter_func(detection) else 'no'
                    filtered.at[i, f'{catalog}_match'] = result

                    if verbose > 1:
                        print(f'{i}: {detection.at["ObsId"]} - {catalog} match: {filtered.at[i, f"{catalog}_match"]}')
                except Exception as e:
                    print(f'{i}: {detection.at["ObsId"]} - {e}')
                    continue
            else:
                if verbose > 1:
                    print(f'{i}: {detection.at["ObsId"]} - {catalog} match: already known.')

    return filtered


def filter_detection_file(detections_filename: str, filtered_filename: str, catalogs: Dict[str, Callable], verbose: int = 0) -> None:
    detections = pd.read_csv(
        detections_filename,
        sep=r"\s+",   # split on one or more spaces
        engine="python",
        names=["ObsId", "RA", "DEC", "THETA", "POS_ERR", "SIGNIFICANCE"],
        header=0,     # skip the first line ("ObsId")
        dtype=str
    )

    filtered = safe_read_filtered(filtered_filename, catalogs, detections)

    if verbose > 0:
        print(f'Analysing {len(detections)} detections from {detections_filename}')

    filtered = filter_detections(detections, filtered, catalogs, verbose)

    filtered.to_csv(filtered_filename, index=False)


def clear_filter_matches(filtered_filename: str, catalog: str) -> None:
    filtered = pd.read_csv(filtered_filename, sep=',', header=0, dtype=str)

    for i, detection in filtered.iterrows():
        if f'{catalog}_match' in filtered.columns:
            filtered.at[i, f'{catalog}_match'] = 'unknown'
    
    filtered.to_csv(filtered_filename, index=False)


def find_new_fxrt(filtered_filename: str, catalogs: Dict[str, Callable], window: str) -> pd.DataFrame:
    filtered = pd.read_csv(filtered_filename, sep=",", header=0, dtype=str, index_col=0)

    catalog_cols = [f"{cat}_match" for cat in catalogs.keys()]

    strict_fxrt = filtered.loc[(filtered[catalog_cols] == "no").all(axis=1)]
    relaxed_fxrt = filtered.loc[~(filtered[catalog_cols] == "yes").any(axis=1)]

    os.makedirs("output/candidates", exist_ok=True)

    strict_fxrt = strict_fxrt.reset_index()
    relaxed_fxrt = relaxed_fxrt.reset_index()

    strict_file = f"output/candidates/new_fxrt_{window}_strict.csv"
    relaxed_file = f"output/candidates/new_fxrt_{window}_possible.csv"

    if len(strict_fxrt) > 0:
        strict_fxrt.to_csv(strict_file, index=False)
        print(f"Saved {len(strict_fxrt)} strict new FXRT candidate(s) to {strict_file}")
    else:
        print("No strict new FXRT candidates found.")

    if len(relaxed_fxrt) > 0:
        relaxed_fxrt.to_csv(relaxed_file, index=False)
        print(f"Saved {len(relaxed_fxrt)} possible FXRT candidate(s) (includes 'unknown') to {relaxed_file}")
    else:
        print("No possible (relaxed) FXRT candidates found.")

    return relaxed_fxrt


WINDOW = "w0.8"

DETECTIONS_FILENAME = f'output/detections_{WINDOW}_forward.txt'
FILTERED_FILENAME   = f'output/filtered_{WINDOW}_forward.csv'

CATALOGS = {
    'gaia': filter_gaia,
    'archival': filter_archival,
    'chandra': filter_chandra,
    'erosita': filter_erosita,
    'ned': filter_ned,
    'simbad': filter_simbad,
    'vizier': filter_vizier,
    'xray-binaries': filter_Xray_binaries
}
VERBOSE = 2


if __name__ == '__main__':
    filter_detection_file(
        DETECTIONS_FILENAME,
        FILTERED_FILENAME,
        CATALOGS,
        VERBOSE
    )

    # find and save new candidates 
    candidates = find_new_fxrt(FILTERED_FILENAME, CATALOGS, WINDOW)

    obsids = pd.read_csv(FILTERED_FILENAME, sep=",", header=0, dtype=str)["ObsId"].unique()
    with open(FILTERED_FILENAME.replace(".csv", ".txt"), "w") as f:
        for obsid in obsids:
            f.write(f"{obsid}\n")

    print(f"Saved ObsId list to {FILTERED_FILENAME.replace('.csv', '.txt')}")


Analysing 1667 detections from output/detections_w0.8_forward.txt
Loaded 1377 total reference detections from 4 files.
0: 17708 - populated from previous filtered files.
1: 17709 - populated from previous filtered files.
2: 25103 - populated from previous filtered files.
3: 25105 - populated from previous filtered files.
INFO: Query finished. [astroquery.utils.tap.core]
4: 25993 - gaia match: no
4: 25993 - archival match: no
4: 25993 - chandra match: no
4: 25993 - erosita match: no
4: 25993 - ned match: no


4: 25993 - 'OTYPES'
4: 25993 - vizier match: no
4: 25993 - xray-binaries match: no
INFO: Query finished. [astroquery.utils.tap.core]
5: 27882 - gaia match: no
5: 27882 - archival match: no
5: 27882 - chandra match: no
5: 27882 - erosita match: no
5: 27882 - ned match: no


5: 27882 - 'OTYPES'
5: 27882 - ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
5: 27882 - xray-binaries match: no
INFO: Query finished. [astroquery.utils.tap.core]
6: 13 - gaia match: yes
6: 13 - archival match: no
6: 13 - chandra match: yes
6: 13 - erosita match: no
6: 13 - ned match: no
6: 13 - 'OTYPES'
6: 13 - vizier match: yes
6: 13 - xray-binaries match: no
7: 366 - populated from previous filtered files.
8: 18 - populated from previous filtered files.
9: 19 - populated from previous filtered files.
10: 21 - populated from previous filtered files.
11: 606 - populated from previous filtered files.
12: 624 - populated from previous filtered files.
13: 634 - populated from previous filtered files.
14: 642 - populated from previous filtered files.
15: 1872 - populated from previous filtered files.
16: 1878 - populated from previous filtered files.
INFO: Query finished. [astroquery.utils.tap.core]
17: 1881 - gaia match: yes
17: 1881 - archiv

33: 4489 - 'OTYPES'
33: 4489 - vizier match: no
33: 4489 - xray-binaries match: no
INFO: Query finished. [astroquery.utils.tap.core]
34: 4491 - gaia match: yes
34: 4491 - ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
34: 4491 - chandra match: no
34: 4491 - erosita match: no
34: 4491 - ned match: yes
34: 4491 - 'OTYPES'
34: 4491 - vizier match: yes
34: 4491 - xray-binaries match: no
35: 4499 - populated from previous filtered files.
36: 5402 - populated from previous filtered files.
37: 5403 - populated from previous filtered files.
INFO: Query finished. [astroquery.utils.tap.core]
38: 5404 - gaia match: no
38: 5404 - archival match: yes
38: 5404 - chandra match: yes
38: 5404 - erosita match: no
38: 5404 - ned match: no
38: 5404 - 'OTYPES'
38: 5404 - vizier match: yes
38: 5404 - xray-binaries match: no
INFO: Query finished. [astroquery.utils.tap.core]
39: 5405 - gaia match: yes
39: 5405 - archival match: yes
39: 5405 - chandra match: yes
39

44: 6122 - 'OTYPES'
44: 6122 - ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
44: 6122 - xray-binaries match: no
45: 6396 - populated from previous filtered files.
46: 7246 - populated from previous filtered files.
47: 7247 - populated from previous filtered files.
48: 6413 - populated from previous filtered files.
INFO: Query finished. [astroquery.utils.tap.core]
49: 6416 - gaia match: yes
49: 6416 - archival match: yes
49: 6416 - chandra match: yes
49: 6416 - erosita match: no
49: 6416 - ned match: no
49: 6416 - 'OTYPES'
49: 6416 - vizier match: yes
49: 6416 - xray-binaries match: no
50: 6417 - populated from previous filtered files.
51: 6418 - populated from previous filtered files.
52: 6419 - populated from previous filtered files.
53: 6436 - populated from previous filtered files.
INFO: Query finished. [astroquery.utils.tap.core]
54: 6437 - gaia match: yes
54: 6437 - archival match: yes
54: 6437 - chandra match: yes
54: 6437 - erosita 

...

1038: 10532 - 'OTYPES'
1038: 10532 - vizier match: no
1038: 10532 - xray-binaries match: no
1039: 10542 - populated from previous filtered files.
1040: 10544 - populated from previous filtered files.
1041: 10545 - populated from previous filtered files.
1042: 11800 - populated from previous filtered files.
INFO: Query finished. [astroquery.utils.tap.core]
1043: 11313 - gaia match: yes
1043: 11313 - archival match: no
1043: 11313 - chandra match: no
1043: 11313 - erosita match: no
1043: 11313 - ned match: no
1043: 11313 - 'OTYPES'
1043: 11313 - ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
1043: 11313 - xray-binaries match: no
1044: 11361 - populated from previous filtered files.
1045: 12951 - populated from previous filtered files.
1046: 12953 - populated from previous filtered files.
INFO: Query finished. [astroquery.utils.tap.core]
1047: 12987 - gaia match: no
1047: 12987 - archival match: yes
1047: 12987 - chandra match: yes
1047: 12987

1054: 13808 - 'OTYPES'
1054: 13808 - vizier match: yes
1054: 13808 - xray-binaries match: no
INFO: Query finished. [astroquery.utils.tap.core]
1055: 13812 - gaia match: no
1055: 13812 - archival match: yes
1055: 13812 - chandra match: yes
1055: 13812 - erosita match: no
1055: 13812 - ned match: yes
1055: 13812 - 'OTYPES'
1055: 13812 - vizier match: no
1055: 13812 - xray-binaries match: yes
1056: 13815 - populated from previous filtered files.
1057: 13816 - populated from previous filtered files.
INFO: Query finished. [astroquery.utils.tap.core]
1058: 15496 - gaia match: no
1058: 15496 - archival match: yes
1058: 15496 - chandra match: yes
1058: 15496 - erosita match: no
1058: 15496 - ned match: no
1058: 15496 - 'OTYPES'
1058: 15496 - vizier match: no
1058: 15496 - xray-binaries match: yes
1059: 15553 - populated from previous filtered files.
INFO: Query finished. [astroquery.utils.tap.core]
1060: 15390 - gaia match: no
1060: 15390 - archival match: no
1060: 15390 - chandra match: no
10

1060: 15390 - 'OTYPES'
1060: 15390 - vizier match: no
1060: 15390 - ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
1061: 16023 - populated from previous filtered files.
INFO: Query finished. [astroquery.utils.tap.core]
1062: 16302 - gaia match: yes
1062: 16302 - archival match: no
1062: 16302 - chandra match: no
1062: 16302 - erosita match: no
1062: 16302 - ned match: no


1062: 16302 - 'OTYPES'
1062: 16302 - vizier match: yes
1062: 16302 - xray-binaries match: no
1063: 16354 - populated from previous filtered files.
INFO: Query finished. [astroquery.utils.tap.core]
1064: 16973 - gaia match: yes
1064: 16973 - ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
1064: 16973 - chandra match: no
1064: 16973 - erosita match: yes
1064: 16973 - ned match: no
1064: 16973 - 'OTYPES'
1064: 16973 - vizier match: no
1064: 16973 - xray-binaries match: no
INFO: Query finished. [astroquery.utils.tap.core]
1065: 18047 - gaia match: no
1065: 18047 - archival match: yes
1065: 18047 - chandra match: yes
1065: 18047 - erosita match: no
1065: 18047 - ned match: yes
1065: 18047 - 'OTYPES'
1065: 18047 - ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
1065: 18047 - xray-binaries match: yes
1066: 18048 - populated from previous filtered files.
INFO: Query finished. [astroquery.utils.tap.core]
1

1067: 19688 - 'OTYPES'
1067: 19688 - vizier match: no
1067: 19688 - xray-binaries match: yes
INFO: Query finished. [astroquery.utils.tap.core]
1068: 18817 - gaia match: no
1068: 18817 - archival match: yes
1068: 18817 - chandra match: yes
1068: 18817 - erosita match: no
1068: 18817 - ned match: yes
